In [15]:
# need to get all  images on perf out of s3, save them, then develop a mapping between the image name and the perf data
import boto3
import os
import pandas as pd
import shutil 
from tqdm.notebook import tqdm
import hashlib

In [4]:
# load in the object_id_to_image_id.csv file
full_mapping = pd.read_csv('/home/jtyo/Repos/PersonalRepos/deep-person-reid/reid-data/performance/object_id_to_image_id.csv')
# now for each image id, get the image from s3

In [5]:
full_mapping

,id,image_id,s3_path
0,1,4,https://labeling-detected-objects-bucket.s3.am...
1,2,3,https://labeling-detected-objects-bucket.s3.am...
2,3,2,https://labeling-detected-objects-bucket.s3.am...
3,4,4,https://labeling-detected-objects-bucket.s3.am...
4,5,2,https://labeling-detected-objects-bucket.s3.am...
...,...,...,...
250414,282507,152227,NaN
250415,282508,152228,NaN
250416,282509,152227,NaN
250417,282510,152230,NaN


In [10]:
s3 = boto3.client("s3")
original_imgs_dir = '/home/jtyo/tmp/mudd_originals'
errors = 0
for s3_url in tqdm(full_mapping['s3_path']):
    try:
        key = s3_url.split('/')[-1]
        if os.path.exists(os.path.join(original_imgs_dir, key)):
            continue
        response = s3.get_object(Bucket='labeling-detected-objects-bucket', Key=key)['Body']
        with open(os.path.join(original_imgs_dir, key), 'wb') as out_file:
                shutil.copyfileobj(response, out_file)
    except Exception as e:
        errors += 1

  0%|          | 0/250419 [00:00<?, ?it/s]

In [11]:
print('errors', errors)

errors 2424


In [12]:
mudd_processed_base_path = '/home/jtyo/Repos/PersonalRepos/deep-person-reid/reid-data/performance'
final_folders = [
    'bounding_box_test', 
    'bounding_box_train',
    'gallery_all',
    'gallery_mud',
    'gallery_no_mud',
    'query',
    'query_all',
    'query_mud',
    'query_no_mud'
]

In [16]:
def get_image_checksum(img_path):
    with open(img_path, 'rb') as f:
        return hashlib.md5(f.read()).hexdigest()

In [17]:
# Now to make things more efficient and quick, lets get a checksum of all images, store in a dict, and then we can easily check if a mudd image exists in the original set
# for every image in original_imgs_dir, compute the checksum and save in memory
original_imgs_checksums = {}
for img in tqdm(os.listdir(original_imgs_dir)):
    original_imgs_checksums[get_image_checksum(os.path.join(original_imgs_dir, img))] = img

  0%|          | 0/247995 [00:00<?, ?it/s]

In [18]:
# now for each image in the mudd_processed_base_path, check if it exists in the original_imgs_checksums
# then we will save a json file that maps the mudd_processed image name to the original image name
mudd_processed_to_original = {}
missed_images = {}
for folder in tqdm(final_folders):
    for img in os.listdir(os.path.join(mudd_processed_base_path, folder)):
        img_checksums = get_image_checksum(os.path.join(mudd_processed_base_path, folder, img))
        if img_checksums in original_imgs_checksums:
            mudd_processed_to_original[os.path.join(folder, img)] = original_imgs_checksums[img_checksums]
        else:
            print(f'Could not find {img} in original images')
            missed_images[img] = img_checksums

print(f'Found {len(mudd_processed_to_original)} images in the original set')
print(f'Missed {len(missed_images)} images in the original set')

  0%|          | 0/9 [00:00<?, ?it/s]

Found 5385 images in the original set
Missed 0 images in the original set


In [22]:
# now save the mapping to a json file
import json
with open(os.path.join(mudd_processed_base_path, 'mudd_processed_to_original.json'), 'w') as f:
    json.dump(mudd_processed_to_original, f)

In [21]:
for i, (k, v) in enumerate(mudd_processed_to_original.items()):
    print(k, v)
    if i > 10:
        break

bounding_box_test/41_91905_1528.png 91905.png
bounding_box_test/25_162073_925.png 162073.png
bounding_box_test/36_266159_3830.png 266159.png
bounding_box_test/41_87791_1438.png 87791.png
bounding_box_test/43_178306_1048.png 178306.png
bounding_box_test/36_266102_3826.png 266102.png
bounding_box_test/36_265909_3806.png 265909.png
bounding_box_test/38_35697_2224.png 35697.png
bounding_box_test/39_177197_1036.png 177197.png
bounding_box_test/39_176626_1032.png 176626.png
bounding_box_test/43_3748_2192.png 3748.png
bounding_box_test/40_93369_1543.png 93369.png


In [23]:
# see if the text 28749223 is anywhere in mudd_processed_to_original 
for k, v in mudd_processed_to_original.items():
    if '28749223' in k or '28749223' in v:
        print(k, v)

bounding_box_train/28749223_131645_845.png 131645.png
bounding_box_train/28749223_131666_850.png 131666.png
bounding_box_train/28749223_131626_842.png 131626.png
bounding_box_train/28749223_27215_29.png 27215.png
bounding_box_train/28749223_26822_26.png 26822.png
bounding_box_train/28749223_133564_852.png 133564.png
bounding_box_train/28749223_131649_849.png 131649.png
bounding_box_train/28749223_26819_25.png 26819.png
bounding_box_train/28749223_131655_843.png 131655.png
bounding_box_train/28749223_131640_848.png 131640.png
bounding_box_train/28749223_131646_847.png 131646.png
bounding_box_train/28749223_1745_2210.png 1745.png
bounding_box_train/28749223_131664_846.png 131664.png
bounding_box_train/28749223_131647_844.png 131647.png
